In [1]:
import sys

sys.path.append('/gpfs/bbp.cscs.ch/project/proj83/home/isbister/physiology_2023/cortexetl/')
sys.path.append('/gpfs/bbp.cscs.ch/project/proj147/home/isbister/spikewarp3/')

import matplotlib.pyplot as plt
import numpy as np
import cortexetl as c_etl
import shared_functions as sf


# pickle_dir = "/gpfs/bbp.cscs.ch/project/proj83/home/isbister/spike_sorting_bias_output_data/pickles/pickles_20-4-24/"
# import os; os.makedirs(pickle_dir, exist_ok=True)

In [ ]:
test_ss_etl_conf_path = "/gpfs/bbp.cscs.ch/project/proj147/home/isbister/experimental_data/1_silico_spike_sorting/4_etl_converted/4_spikesorting_stimulus_test_neuropixels_8-1-24__8slc_80f_360r_50t_200ms_1_smallest_fiber_gids/KS2/ss/analysis_config_cond_windows.yaml"
window_prefix = "evoked_SOZ_25ms_"

ma = c_etl.analysis_initial_processing(test_ss_etl_conf_path, create_figure_dirs=False)
a = ma.array0

In [21]:
import pandas as pd
window_info_df = pd.DataFrame(ma.global_config.custom["window_info_dict"]).T
window_info_df = window_info_df.rename_axis('window')
stim_windows_df = pd.merge(a.repo.windows.df, window_info_df, on=['window'])

In [ ]:
def collate_and_prepare_rotation_spike_sorted_data(a):
    
    # Windows
    window_prefix = "evoked_SOZ_25ms_"
    rotation_windows = [i for i in list(a.repo.windows.df.window.unique()) if window_prefix in i]
    a.repo.windows.df["rotation"] = a.repo.windows.df.apply(lambda row : float(row["window"].split("_")[-1]) if row["window"] in rotation_windows else np.nan, axis=1)
    a.repo.windows.df["rotation_radians"] = sf.deg_to_rad(a.repo.windows.df["rotation"])
    
    # Features, spikes and histograms
    neuron_classes = ['all']
    data_with_rotation_info = sf.process_features_and_spikes(a, rotation_windows, neuron_classes)
    
    return data_with_rotation_info
    
data_with_rotation_info = collate_and_prepare_rotation_spike_sorted_data(a)

In [ ]:
# data_with_rotation_info.keys()

In [ ]:
import pickle
with open(pickle_dir + 'data_with_rotation_info_spike_sorted.pickle', 'wb') as handle:
    pickle.dump(data_with_rotation_info, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open(pickle_dir + 'data_with_rotation_info_spike_sorted.pickle', 'rb') as handle:
    data_with_rotation_info = pickle.load(handle)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def polar_tuning_curve_plots(features_by_spiking_gid_rotation_windows, neuron_classes, stat_key, mean_spike_count_threshold=-1):
    for neuron_class in neuron_classes:
        for i in np.unique(features_by_spiking_gid_rotation_windows.etl.q(neuron_class=neuron_class)['spiking_neuron_class_index']):
            n_df = features_by_spiking_gid_rotation_windows.etl.q(spiking_neuron_class_index=i, neuron_class=neuron_class)

#             n_df = n_df[n_df['mean_spike_counts'] > mean_spike_count_threshold] 

            n_df.sort_values(by=['rotation'], inplace=True)
    
            if len(n_df) == 40:

                r = n_df[stat_key]
                theta = n_df['rotation_radians']

                fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
#                 ax.plot(theta, r)
                ax.scatter(theta, r)
                ax.set_rmax(r.max())
                # ax.set_rticks([0.5, 1, 1.5, 2])  # Less radial ticks
                ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
                ax.grid(True)

                ax.set_title("A line plot on a polar axis", va='bottom')
#                 plt.savefig(str(n_df['spiking_neuron_class_index'].iloc[0]))
                plt.show()
                plt.close()

# polar_tuning_curve_plots(data_with_rotation_info['features_by_spiking_gid_rotation_windows'], ['all'], 'mean_spike_counts', mean_spike_count_threshold=-1)
polar_tuning_curve_plots(data_with_rotation_info['features_by_spiking_gid_rotation_windows'], ['all'], 'first_spike_time_means_cort_zeroed', mean_spike_count_threshold=-1)

In [2]:
import pandas as pd


,true_gid,0,1,2,3,4,5,6,7,8,...,2613,2614,2615,2616,2617,2618,2619,2620,2621,2622
0,12165,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
1,15894,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
2,16652,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
3,17923,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
4,18371,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,4223302,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
1832,4225319,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000
1833,4228700,0.000313,0.000294,0.0,0.0,0.000591,0.0,0.0,0.000404,0.0,...,0.0,0.000169,0.0,0.00000,0.0,0.0,0.00017,0.0,0.0,0.000146
1834,4229218,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000


In [ ]:
# For each gid plot rotation vs first spike for each trial
for gid in np.unique(data_with_rotation_info['features_by_gid_and_trial_rotation_windows'].etl.q(neuron_class="all")['gid']):
    
    plt.figure()
    n_features_by_gid_and_trial_rotation_windows = data_with_rotation_info['features_by_gid_and_trial_rotation_windows'].etl.q(gid=gid)
    plt.scatter(n_features_by_gid_and_trial_rotation_windows['rotation'], n_features_by_gid_and_trial_rotation_windows['first'])
    plt.show()
    plt.close()

In [ ]:
w1_spikes = a.repo.report.df.etl.q(window='w1')

rotation = 20
import math 
sim_id = math.floor(rotation / 10)

print(sim_id)
start_time = (sim_id + 1) * 500 + rotation * 10000
end_time = start_time + 1000

import matplotlib.pyplot as plt
plt.figure(figsize=(20, 6))
plt.scatter(w1_spikes['time'], w1_spikes['gid'], s=0.8)
plt.gca().set_xlim([start_time, end_time])
plt.show()
plt.close()

In [ ]:
rot_spikes = a.repo.report.df.etl.q(window='evoked_SOZ_25ms_0.0')

In [ ]:
rot_features = a.features.by_gid_and_trial.df.etl.q(window=['evoked_SOZ_25ms_0.0', 'evoked_SOZ_25ms_1.0', 'evoked_SOZ_25ms_2.0', 'evoked_SOZ_25ms_3.0', 'evoked_SOZ_25ms_4.0', 'evoked_SOZ_25ms_5.0', 'evoked_SOZ_25ms_6.0', 'evoked_SOZ_25ms_7.0', 'evoked_SOZ_25ms_8.0']).reset_index() # window='evoked_SOZ_25ms_0.0'

In [ ]:
import numpy as np


rot_features['did_spike'] = rot_features['first'].notna()
did_spike_counts = rot_features.groupby(['gid']).sum()['did_spike']

for gid in did_spike_counts[did_spike_counts > 10].index:
    print(gid)
    gid_rot_features = rot_features.etl.q(gid=gid)
    
    plt.figure()
    plt.scatter(gid_rot_features['trial'], gid_rot_features['first'])
    plt.show()
    plt.close()
    
    


In [ ]:
import math

rotations = np.arange(0.0, 360.0, 1.0)
# rotations = np.arange(0.0, 6.0, 1.0)
# print(rotations)

# initial_offset = 500
n_trials = 50

trial_steps_value = 200
window_type = "evoked_stimulus_onset_zeroed"
windows = ["evoked_SOZ_25ms"]
boundss = [[0, 25]]

test_str = f""
for rotation_index, rotation in enumerate(rotations):
    
    sim_id = math.floor(rotation / 10)
    
    for window, bounds in zip(windows, boundss):
        
#         rotation_offset = initial_offset + (sim_id * 1000.0) + rotation_index * n_trials * trial_steps_value
        rotation_offset = ((sim_id + 1) * 500) + rotation * float(10000)

        window_name = window + "_" + str(rotation)
        test_str += f"        {window_name}:\n          bounds: {bounds}\n          initial_offset: {rotation_offset}\n          n_trials: {n_trials}\n          trial_steps_value: {trial_steps_value}\n          window_type: {window_type}\n\n"

# print(test_str)

content = test_str
text_file = open("rotation_windows.txt", "w")
n = text_file.write(content)
text_file.close()